<img style="float: left;" src="Hda_logo_res.png">

---

# __<p style="text-align: center;">Digitalisierung: Grundlagen und Praxis</p>__
### <p style="text-align: center;">Prof. Dr. Elke Lang</p>
### <p style="text-align: center;">Prof. Dr. Stefan Schmunk</p>
## __<p style="text-align: center;">Bomber's Baedeker: automatisierte Umwandlung zu XML</p>__
### <p style="text-align: center;">Felix Bach (754708) und Cristian Secco (748086)</p>
### <p style="text-align: center;">Informationswissenschaft B. Sc. WS2019/2020</p>
---


## Inhaltsverzeichnis

* __I Imports & Dateien__


* __1. Zusatzfunktionen__
        
    
* __2. Preprocessing__
    * 2.1. Städteliste von Open Street Maps
    

* __3. Funktionen__
    * 3.1. Herauslesen der Städtenamen
        * 3.1.1. Herauslesen der Städtenamen verfeinerte Version
        * 3.1.2. Abgleich der Ergebnisse mit der OSM-Städteliste
    * 3.2. Herauslesen der Bundesländer
    * 3.3. Herauslesen der Geokoordinaten
    * 3.4. Herauslesen der Entfernung zu London
    * 3.5. Herauslesen der Einwohnerzahl
        
        
* __4. Export__
    * 4.1. XML-Export
    * 4.2. CSV-Export

---
## I Imports und Dateien

In [605]:
#Alle in diesem Notebook verwendete Bibliotheken
import sys
import os
import os.path
#Verarbeiten von CSV-Dateien
import csv
#Verarbeiten von XML-Dateien
from xml.etree import ElementTree as ET
#Reguläre Ausdrücke
import re

Folgende Dateien werden für diese Notebook benötigt:
* Hda_logo_res.png: Das Logo der Hochschule Darmstadt
* bomber_ocr_final.txt : Die OCR von The Bomber's Baedeker
    * Quelle des Original PDFs (wurde von uns bearbeitet): https://visualcollections.ub.uni-mainz.de/histbuch/content/pageview/454132
* staedte_osm.txt : Die Open Street Maps Städteliste
    * Quelle: https://www.datendieter.de/item/Liste_von_deutschen_Staedtenamen_.csv
    * Daten von OpenStreetMap (http://www.openstreetmap.org/) - Veröffentlicht unter CC-BY-SA 2.0 (http://creativecommons.org/licenses/by-sa/2.0/) Lizenz. Jede Ableitung der Daten muss auch unter dieser Lizenz stehen. Bei jeder Verwendung muss obiger Herkunftsnachweis mitangegeben werden.
* bomber_xml: XML Datei, die als Vorlage für die Ergebnisse dient

Folgende Dateien werden von diesem Notebook erstellt:
* bomber_output.xml: XML-Datei mit den Ergebnissen
* bomber.csv: CSV-Datei für die Verwendung mit dem DARIAH Geobrowser

---

## 1. Zusatz-Funktionen

### Funktion zum Einlesen des Textes
Mit dieser Funktion kann eine Text-Datei eingelesen, und in einer Variable gespeichert werden.

In [618]:
def file_open(textfile):
    
    #Importieren der Datei
    try:
        fp = open(textfile)
        
    #Wenn die Datei nicht gefunden wird    
    except:        
        print("File not found")
        
    else:
        
        #Speichern der Datei in einer Variablen
        text = fp.read()
        
        return text

## 2. Preprocessing

Vor der eigentlichen Verarbeitung wird der Text in einer Variable gespeichert. Ausserdem wird ein Versuch unternommen die OCR zu korrigieren. Weitere Korrekturen wurden dann aber von Hand vorgenommen, da es sich nicht gelohnt hat, für jeden möglichen OCR-Fehler eine eigene Abfrage zu schreiben.

In [665]:
#Öffnen und Speichern des Textes in einer Variable
bomber_text_raw=file_open("bomber_ocr_final.txt")

#000 wurde oft als OCX klassifiziert und wird hier automatisch ersetzt
bomber_text=(bomber_text_raw.replace("OCX","000"))

#Ausgabe des Textes als Test
print(bomber_text)

THE
BOMBER'S BAEDEKER
(Guide to the Economic Importance of German Towns and Cities)
2nd (1944) Edition.
Part I.
AACHEN-KÜSTRIN

Enemy Branch (Foreign Oilieo & Ministry of Economic Warfare)
THE BOMBER'S BAEDEKER
(Guide to the Economic Importance of German Towns & C ties) 2nd (1944) Edition.
PART I
AACHEN-KÜSTRIN

Enemy Branch,
(Foreign Office & Ministry of Economic Warfare).
PREPACK
1,The 1944 edition of the "Bcmber's Baed revised and amplified version of Part III edition,which it replaces. It has been scope to cover all towns in Germany (re which are of any industrial significance, and it has been possible to expand the number of industrial establishments listed in each town. Information relating to the activities of particular factories has been revised in the light of the latest available intelligence.
2.As in the previous edition, enterprises have been grouped under fourteen industrial headings as follows:-
1.Transportation
2.Public Utility Services
3.Solid Fuels
4.Liquid Riels & Su

---

### 2.1. Städteliste von Open Street Maps

Da wird bei den ersten Versuchen eine große Masse an Ergebnissen bekommen haben, kamen wir auf die Idee Städte zur Korrektur mit einer Städteliste von Open Street Maps abzugleichen.

Über einen langen Zeitraum der Entwicklung haben wir an diesem Verfahren festgehalten, um unsere Ergebnisse korrekt zu halten. Im späteren Verlauf kamen wir allerdings zu dem Punkt, dass die OCR von Hand so gut korrigiert wurde, dass wir schon vor dem Abgleich eine geringe Fehlerrate hatten. 

Zusätzlich hatten wir das Problem, dass auf Grund der Aktualität der Open Street Maps Liste viele Städte aus der damaligen Zeit in dieser nicht enthalten waren (und daher herausgefiltert wurden).

Letztenendes haben wir uns dazu entschlossen, den Abgleich mit der Städteliste wieder wegzulassen, was zu besseren Ergebnissen geführt hat. Allerdings auch nur, weil die OCR manuell aufbereitet wurde.

In [686]:
#Öffnen der Open Street Maps Datei
cities_osm_raw=file_open("staedte_osm.txt")

#Umwandeln der Einträge in Großbuchstaben (da Städtenamen im Bomber's Baedeker nur 
#aus Großbuchstaben bestehen)
cities_osm_temp=cities_osm_raw.upper()

#Liste der Städte
cities_osm=[]

#Aufteilen der Städte in einzelne Listenelemente 
for line in cities_osm_temp.splitlines():
    
    temp=line.replace('"','')
    
    cities_osm.append(temp)    
    
#Ergebnis
print(cities_osm)

['AACH', 'AACHA', 'AACHEN', 'AALEN', 'AALEN-ATTENHOFEN', 'AALEN-EBNAT', 'AALEN-ESSINGEN', 'AALEN-FACHSENFELD', 'AALEN-UNTERKOCHEN', 'AALEN-UNTERROMBACH', 'AARBERGEN', 'AARBERGEN-MICHELBACH', 'AASEN', 'ABBENDORF', 'ABBERODE', 'ABENBERG', 'ABENDEN', 'ABENS', 'ABENSBERG', 'ABENSBERG GADEN', 'ABENSBERG OFFENSTETTEN', 'ABENSBERG SANDHARLANDEN', 'ABERSFELD', 'ABSBERG', 'ABSTATT', 'ABSTEINACH', 'ABSTETTERHOF', 'ABTSGMÜND', 'ABTSTEINACH', 'ABTSWIND', 'ACH', 'ACHBERG', 'ACHENBACH', 'ACHERN', 'ACHERN/GAMSHURST', 'ACHERN-MÖSBACH', 'ACHERN-SASBACHRIED', 'ACHIM', 'ACHIM-BADEN', 'ACHTERWEHR', 'ACHTRUP', 'ADELBERG', 'ADELEBSEN', 'ADELHEIDSDORF', 'ADELMANNSFELDEN', 'ADELSDORF', 'ADELSDORF-NEUHAUS', 'ADELSHEIM', 'ADELSHOFEN', 'ADELSTETTEN', 'ADELZHAUSEN', 'ADENAU', 'ADENDORF', 'ADERSTEDT', 'ADLERSBERG', 'ADLKOFEN', 'ADMANNSHAGEN', 'ADOLZHAUSEN', 'ADORF', 'ADORF/VOGTL', 'AERZEN', 'AERZEN-GROSS-BERKEL', 'AFFALTERBACH', 'AFFECKING', 'AFFING', 'AFFINGHAUSEN', 'AFFING-MÜHLHAUSEN', 'AFTHOLDERBERG', 'AGATHARI

---

## 3. Funktionen

### 3.1 Herauslesen der Städtenamen

Diese Funktion sucht im Text nach Städtenamen, diese bestehen (nach unserer eigenen Konvention) nur aus Großbuchstaben und sind mindestens 3 Zeichen lang.

In [666]:
#Funktion die alle Begriffe speichert, die nur aus mindestens 3 Großbuchstaben bestehen
def find_cities (text):
    
    city_names=[]
        
    i=1
    
    #Durchlaufen der einzelnen Zeilen
    for line in text.splitlines():
        
        #Überspringen der Titel- und Innenseiten
        if i > 56:
            
            #Regulärer Ausdruck den gesuchten Begriff
            city_name=re.findall(r'[A-Z_\s]{3,}', line)
            
            #Filtern von Duplikaten
            if city_name and city_name not in city_names:
                
                city_names.append(city_name)
                
        else:
            
            i+=1
    
    
    city_names_clean=[]
    
    #Säubern der Ergebnisse
    for city in city_names:
        
        #Herausnehmen von Leerzeichen
        clean_city=str(city[0]).replace(' ','')
        
        #Nur Begriffe die jetzt immernoch aus min. 3 und höchstens 11 Zeichen bestehen werden gespeichert
        if len(clean_city) > 2 and len(clean_city) <20:
            
            city_names_clean.append(clean_city)
    
    
    return city_names_clean

In [668]:
#Ergebnis
result=find_cities(bomber_text)
print(result)

['AACHEN', 'LOCATION', 'PRIORITYRATING', 'AALEN', 'AHLEN', 'AKEN', 'ALBBRUCK', 'ALLENDORF', 'DYNAMITA', 'ALLENSTEIN', 'ALSDORF', 'ALTDAMM', 'ALTENA', 'ALTENBURG', 'ALT', 'AMBERG', 'AMMENDORF', 'ANDERBACH', 'ANDREASH', 'VOHAG', 'VOHAG', 'ANKLAM', 'ANNABERG', 'ANSBACH', 'NIMSWAI', 'ARNSBERG', 'ARNSTADT', 'ASCHAFFENBURG', 'ASCHERSLEBEN', 'KZL', 'AUE', 'NON', 'AUERBACH', 'AUGSBURG', 'UOU', 'AUGUSTENTHAL', 'BADEN', 'BAD', 'BAD', 'BAD', 'BAD', 'BAMBERG', 'BARMEN', 'BASDORF', 'BAUTZEN', 'BAYREUTH', 'BELGRAD', 'BENRATH', 'BENSBERG', 'BENSHEIM', 'BERGEDORF', 'BERGHEIM', 'VAWAG', 'BERGISCH', 'BERLIN', 'EWAG', 'CRH', 'BERLIN', 'ELECTRICITYSUPPLY', 'BKLB', 'YAG', 'EWAG', 'MEWM', 'ESA', 'EWAG', 'EWAG', 'MEW', 'STEW', 'BKL', 'BKLBE', 'MEW', 'BKLBEWAG', 'BEWAG', 'BKLBEWA', 'BKLBEWAG', 'BKLBEWAG', 'BKLHEY', 'BKLBEY', 'BKL', 'BKLBEWAG', 'BKLBSWAG', 'BKLBEWAG', 'VAG', 'WAG', 'KLBEWAG', 'EKLBEWAG', 'ESA', 'IIGASSUPPLYG', 'GASAG', 'III', 'KOEPENICKERSTR', 'FRIEDENAU', 'SEMENS', 'BOSCH', 'GUSIICALSA', 'BER

In [610]:
#Anzahl:
len(result)

403

Mit einer Anzahl von 403 Ergebnisse, haben wir hier natürlich viel zu viele Werte als korrekt geliefert bekommen. Wie man in der Ausgabe sehen kann, gibt es auch Einträge wie "III".
Für eine bessere Suche, bedarf es daher einer Anpassung der Funktion:

#### 3.1.1. Herauslesen der Städtenamen verfeinerte Version

Diese Funktion liest zusätzlich zu der Zeile des Städtenamens die darauffolgende Zeile und prüft ob diese Koordinaten enthält.


Der reguläre Ausdruck für Städtenamen wurde angepasst: Eine Stadt muss nun mit dem Buchstaben A-K anfangen (da wir nur mit dem 1. Band des Bomber's Baedeker gearbeitet haben), aus Großbuchstaben bestehen, darf Leerzeichen und als Sonderzeichen höchstens ein "-" enthalten. Ausserdem können auch Umlaute vorkommen und die Zeichenlänge wurde auf 14 Zeichen erhöht.

Die Prüfung auf Koordinaten erfolgt mit der simplen Abfrage ob "miles:" in der Zeile nach dem potentiellen Stadtnamen vorkommt. Für eine weitere Ausarbeitung würde es sich empfehlen auch hier möglicherweise mit einem, genaueren, regulären Ausdruck zu arbeiten, welcher die Muster der Koordinaten erkennt.

In [675]:
def find_cities_detailed (text):
    
    #Liste der Städtenamen
    city_names=[]
    
    #Check Variable für korrekte Ergebnisse
    is_valid=False
    
    #Aufsplitten des Textes in Zeilen
    text_split=text.splitlines()
    
    #Hilfsvariablen
    i=1
    
    #Die Zeile nach der aktuellen Zeile
    next_line= None
    
    #Durchlaufen des Textes
    for index, line in enumerate(text_split):
        
        #Überspringen der Titel- und Innenseite        
        if i > 56:
            
            #Zwischenspeichern der nächsten Zeile
            if index < (len(text_split)-1):
                
                next_line = text_split[index+1]
            
            #Zwischenspeichern des Stadtnamens
            city_name_temp=re.findall(r'^[A-K][A-ZÄÖÜ_\s-]{3,}', line)
            
            #Check ob Koordinaten vorhanden sind
            if "miles:" in next_line: 
                
                is_valid=True
                       
            #Speichern des Städtenamens wenn Koordinaten gefunden wurden
            if city_name_temp and city_name_temp not in city_names and is_valid==True:

                city_names.append(city_name_temp)
                
                #Zurücksetzen der Check-Variable
                is_valid=False

        
        else:
            i+=1
    
    #Säubern der Ergebnisse
    city_names_clean_detailed=[]

    for city in city_names:
        
        #Herausnehmen von Leerzeichen
        clean_city=str(city[0]).replace(' ','')
        
        #Nur Begriffe die jetzt immernoch aus min. 3 Zeichen bestehen werden gespeichert
        if clean_city not in city_names_clean_detailed and len(clean_city)>2:
            
            city_names_clean_detailed.append(clean_city)
    
    return city_names_clean_detailed

In [676]:
#Ergebnis:
result_detailed=find_cities_detailed(bomber_text)
print(result_detailed)
len(result_detailed)

['AACHEN', 'AALEN', 'AHLEN', 'AKEN', 'ALBBRUCK', 'ALLENDORF', 'ALLENSTEIN', 'ALSDORF', 'ALTDAMM', 'ALTENA', 'ALTENBURG', 'ALTÖTTING', 'AMBERG', 'AMMENDORF', 'ANDERBACH', 'ANDREASHÜTTE', 'ANKLAM', 'ANNABERG', 'ANSBACH', 'ARNSBERG', 'ARNSTADT', 'ASCHAFFENBURG', 'ASCHERSLEBEN', 'AUE', 'AUERBACH', 'AUGSBURG', 'AUGUSTENTHAL', 'BADEN-BADEN', 'BAD-GODESBERG', 'BAD-HOMBURG', 'BAD-KREUZNACH', 'BAD-OEYNHAUSEN', 'BAMBERG', 'BASDORF', 'BAUTZEN', 'BAYREUTH', 'BELGRAD', 'BENSBERG', 'BENSHEIM', 'BERGEDORF', 'BERGHEIM', 'BERGISCH-GLADBACH', 'BERLIN', 'BERNBURG', 'BETZDORF-SIEGTHAL', 'BEUEL', 'BEUTHEN', 'BIELEFELD', 'BINGEN', 'BITTERFELD', 'DESCHOWITZ', 'BOCHOLT', 'BOCHUM', 'BÖBLINGEN', 'BONN', 'BOTTROP', 'BRACKWEDE', 'BRANDENBURG', 'BRAUNSBERG', 'BREMEN', 'BREMERHAVEN', 'BRESLAU', 'BRUCHSAL', 'BRÜHL', 'BUNZLAU', 'BURG', 'BURGHAUSEN', 'BURGSTADT', 'CASTROP-RAUXEL', 'CELLE', 'CHEMNITZ', 'CLAUSTHAL-ZELLERFELD', 'COBURG', 'COSWIG-ANHALT', 'COSWIG', 'COTTBUS', 'CRIMMTSCHAU', 'CUXHAVEN', 'DANZIG', 'DARMSTAD

219

Im Vergleich zu den vorherigen 403 Ergebnissen, haben wir jetzt mit 189 ein wesentlich kleineren Datensatz. Allerdings besteht dieser nur noch aus korrekten Einträgen.

#### 3.1.2. Abgleich der Ergebnisse mit der OSM-Städteliste

Diese Funktion wurde ursprünglich benutzt um die Ergebnisse zu filtern, wird aber nach einer manuellen Korrektur der OCR nicht mehr benötigt.

In [677]:
#Funktion die ermittelt, ob ein gefundener Stadtname in der OSM-Liste auftaucht
def filter_cities(cities_osm):
    
    #Herauslesen der Städtenamen
    city_names=find_cities_detailed(bomber_text)
    
    set1=set(city_names)
    
    
    set2=set(cities_osm)
    
    #Vergleich   
    filtered_cities=set1.intersection(set2)

            
    return filtered_cities
        

In [678]:
#Ergebnis
result_filtered=sorted(filter_cities(cities_osm))
print(result_filtered)
len(result_filtered)

['AACHEN', 'AALEN', 'AHLEN', 'AKEN', 'ALBBRUCK', 'ALLENDORF', 'ALSDORF', 'ALTENA', 'ALTENBURG', 'ALTÖTTING', 'AMBERG', 'ANKLAM', 'ANSBACH', 'ARNSBERG', 'ARNSTADT', 'ASCHAFFENBURG', 'ASCHERSLEBEN', 'AUE', 'AUERBACH', 'AUGSBURG', 'BADEN-BADEN', 'BAMBERG', 'BAUTZEN', 'BAYREUTH', 'BENSHEIM', 'BERGHEIM', 'BERGISCH-GLADBACH', 'BERLIN', 'BERNBURG', 'BIELEFELD', 'BINGEN', 'BITTERFELD', 'BOCHOLT', 'BOCHUM', 'BONN', 'BOTTROP', 'BRANDENBURG', 'BREMEN', 'BREMERHAVEN', 'BRUCHSAL', 'BRÜHL', 'BURG', 'BURGHAUSEN', 'BÖBLINGEN', 'CASTROP-RAUXEL', 'CELLE', 'CHEMNITZ', 'CLAUSTHAL-ZELLERFELD', 'COBURG', 'COSWIG', 'COTTBUS', 'CUXHAVEN', 'DARMSTADT', 'DATTELN', 'DEMMIN', 'DESSAU', 'DETMOLD', 'DILLINGEN', 'DORMAGEN', 'DORTMUND', 'DREETZ', 'DRESDEN', 'DUDWEILER', 'DUISBURG', 'DÜLKEN', 'DÜREN', 'DÜSSELDORF', 'EILENBURG', 'EISENACH', 'EISENBERG', 'EISLEBEN', 'ELMSHORN', 'EMDEN', 'EMMENDINGEN', 'EMMERICH', 'EMSDETTEN', 'ERFURT', 'ERLANGEN', 'ESCHWEGE', 'ESCHWEILER', 'ESSEN', 'EUSKIRCHEN', 'FALKENSEE', 'FALKENSTEI

159

Hier sieht man sehr gut, dass der Abgleich nun eher schadet: Es werden von 189 Städten nur noch 138 als korrekt gewertet, die 51 fehlenden sind höchstwahrscheinlich heute nicht mehr Teil Deutschlands.

---
### 3.2 Herauslesen der Bundesländer

Im nächsten Schritt wird auf Basis der bereits erstellten Städteliste das Dokument nach den jeweiligen Bundesländern durchsucht. 

Ein Bundesland wurde von uns definiert als der Wert, welcher hinter einer Stadt in Klammern angegeben ist.
Ausnahmen bilden Städte wie Berlin, Bremen, etc. bei diesen wurde der Stadtname als Bundeslandname eingetragen.

Problematisch sind hier Einträge aus dem Original-Text, bei denen von dieser Formatierung abgewichen wurde, daher kann es sein, dass der Bundesland-Wert bei manchen Städten kein Bundesland enthält, sonder Text wie "inklusive Stadtteil XY".

Im Abgleich der Städtenamen musste in dieser (und den folgenden) Funktion(en) darauf geachtet werden, dass Städtenamen keine anderen Städtenamen enthalten (Beispiel: DUISBURG und BURG), sonst kommt es zu Doppeleinträgen.
Daher wird eine Zeile mit einer Stadt durch die Leerzeichen aufgeteilt und das erste Element muss zu 100% mit dem Städtenamen übereinstimmen.


Beispiel: 
* DUISBURG (RHINELAND) würde so DUISBURG und BURG enthalten.
* -> Aufteilung in die Elemente DUISBURG und (RHINELAND)
* Nun wird DUISBURG auf 100% Übereinstimmung geprüft und kann so keine Doppelergebnisse liefern

In [557]:
#Funktion die Namen der Bundesländer speichert
def find_city_state (text):
    
    #Liste der Städtenamen
    city_names=find_cities_detailed(bomber_text)
        
    #Liste der Bundesländer
    state_list=[]
    
    #Aufsplitten des Textes in Zeilen
    text_split=text.splitlines()
    
    #Hilfsvariablen
    i=1
    
    #Durchlaufen des Textes
    for index, line in enumerate(text_split):
        
        #Überspringen der Titel- und Innenseite        
        if i > 56:
            
            #Durchlaufen der Städteliste
            for city in city_names:
                
                #Überprüfen ob das erste Element der Zeile komplett mit dem Stadtnamen übereinstimmt
                if city == line.split(" ")[0]:                    
                    
                    #Check ob in der nächsten Zeile Koordinaten vorkommen
                    #Zeile muss kürzer als 50 Zeichen sein, um Fließtext auszuschließen
                    if "miles:" in text_split[index+1] and len(text_split[index+1]) < 50:
                        
                        #Erneute Überprüfung ob die nächste Zeile Koordinaten enthält
                        if bool(re.search(r'[°]',  text_split[index+1])):
                            
                            #Ausplitten der gefundenen Zeile an der öffnenen Klammer
                            split=line.split("(")
                            
                            #Abfangen von Städten die kein Bundesland angegeben haben
                            try:
                                
                                #Wenn es ein Bundesland gibt
                                state = split[1]
                                
                                #wird es ohne die schließende Klammer an die Ergebnisliste angefügt
                                state_list.append(state.strip(')'))
                            
                            #gibt es kein Bundesland
                            except IndexError:
                                
                                #wird der Stadtname als Bundesland eingetragen
                                state_list.append(split[0])                             
                            

                                  
        else:
            i+=1
    
    return state_list

In [687]:
len(find_city_state(bomber_text))

219

Durch das Abfangen von Fehlern und Ersetzen von Werten, kommen wir auf die gleiche Anzahl an Ergebnissen, wie wir Städtenamen haben. Dies ist wichtig, da alle Ergebnislisten gleich lang sein müssen, um eine korrekte Zuordnung der Werte in der XML-Datei zu gewährleisten.

---
### 3.3 Herauslesen der Geokoordinaten
Mit dieser Funktion können die Geokoordinaten einer Stadt gespeichert werden.

Geokoordinaten kommen immer in einer Zeile nach einem Städtenamen vor. Diese Zeile wird dann, sofern sie als "Koordinaten-Zeile" erkannt wurde, an den ":" aufgeteilt (in die 3 Elemente Koordinaten, Entfernung und Einwohnerzahl) und das erste Element als Koordinaten gespeichert.

Vor der Speicherung werden die Koordinaten noch vom Format "51° N. 12° 25' E" in "51.,12.25" umgewandelt, um im DARIAH Geobrowser verwendet werden zu können.

In [680]:
#Funktion die Geokoordinaten speichert
def find_city_coordinates (text):
    
    #Liste der Städtenamen
    city_names=find_cities_detailed(bomber_text)
    
    #Liste der Koordinaten
    coordinates_list=[]
    
    #Aufsplitten des Textes in Zeilen
    text_split=text.splitlines()
        
    #Hilfsvariablen
    i=1
    
    #Durchlaufen des Textes
    for index, line in enumerate(text_split):
        
        #Überspringen der Titel- und Innenseite        
        if i > 56:
            
            #Durchlaufen der Städteliste
            for city in city_names:               
                
                #Überprüfen ob das erste Element der Zeile komplett mit dem Stadtnamen übereinstimmt           
                if city == line.split(" ")[0]:
                    
                    #Check ob in der nächsten Zeile Koordinaten vorkommen
                    #Zeile muss kürzer als 50 Zeichen sein, um Fließtext auszuschließen                                      
                    if "miles:" in text_split[index+1] and len(text_split[index+1]) < 50:
                                                                        
                        #Erneute Überprüfung ob die nächste Zeile Koordinaten enthält
                        if bool(re.search(r'[°]',  text_split[index+1])):  
                            
                            #Auswahl der Koordinatenzeile
                            next_line=(text_split[index+1])
                            
                            #Aufteilen der Zeile in 3 Elemente anhand der Doppelpunkte und Auswahl des ersten Objekts
                            temp=next_line.split(":")[0]
                            
                            #Umformen in neues Koordinatenformat
                            temp=temp.replace("°",".")
                            temp=temp.replace("'", "")
                            temp=temp.replace("E", "")
                            temp=temp.replace(".E", "")
                            temp=temp.replace("N.", ",")
                            temp=temp.replace(".N", ",")
                            temp=temp.replace(" ","")                            
                            final=temp.replace("' E","")
                                                        
                            #Speichern der Koordinaten
                            coordinates_list.append(final)
                    
        else:
            i+=1
            
    return coordinates_list

In [681]:
len(find_city_coordinates(bomber_text))

219

---
### 3.3 Herauslesen der Entfernung zu London

Die nächste Funktion extrahiert die, in Meilen angegebene, Entfernung der Zielstadt zu London. Diese taucht immer in einer Koordinatenzeile auf und steht an der 2. Position wenn die Zeile an den Doppelpunkten getrennt wird.

In [626]:
#Funktion um die Entfernung zu London zu speichern
def find_city_distances (text):
    
    #Liste der Städtenamen
    city_names=find_cities_detailed(bomber_text)
    
    #Liste der Entfernungen zu London
    distances_list=[]
    
    #Aufsplitten des Textes in Zeilen
    text_split=text.splitlines()
    
    #Hilfsvariablen
    i=1
    
    #Durchlaufen des Textes
    for index, line in enumerate(text_split):
        
        #Überspringen der Titel- und Innenseite        
        if i > 56:
            
            #Durchlaufen der Städteliste
            for city in city_names:
                
                #Überprüfen ob das erste Element der Zeile komplett mit dem Stadtnamen übereinstimmt    
                if city == line.split(" ")[0]:                    
                    
                    #Check ob in der nächsten Zeile Koordinaten vorkommen
                    #Zeile muss kürzer als 50 Zeichen sein, um Fließtext auszuschließen                                        
                    if "miles:" in text_split[index+1] and len(text_split[index+1]) < 50:
                        
                        #Erneute Überprüfung ob die nächste Zeile Koordinaten enthält
                        if bool(re.search(r'[°]',  text_split[index+1])):
                            
                            #Auswahl der Koordinatenzeile
                            next_line=(text_split[index+1])
                            
                            #Aufteilen der Zeile in 3 Elemente anhand der Doppelpunkte und Auswahl des zweiten Objekts
                            distance=next_line.split(":")[1]
                            
                            #Anhängen der Entfernung an die Ergebnisliste
                            distances_list.append(distance)
                            
                                                
        else:
            i+=1
            
    return distances_list

In [682]:
len(find_city_distances(bomber_text))

219

---
### 3.3 Herauslesen der Einwohnerzahl

Als letzte Funktion gibt es die Möglichkeit, die Einwohnerzahl einer Stadt zu speichern. Diese steht in der Koordinatenzeile immer als letztes Element und in Klammern. Die Klammern werden vor dem Speichern entfernt und das Dezimalkomma in einen Dezimalpunkt umgewandelt.

In [628]:
#Funktion um die Einwohnerzahl zu speichern
def find_city_population (text):
    
    #Liste der Städtenamen
    city_names=find_cities_detailed(bomber_text)
    
    #Liste der Einwohnerzahlen
    population_list=[]
    
    #Aufsplitten des Textes in Zeilen
    text_split=text.splitlines()
    
    #Hilfsvariablen
    i=1
    
    #Durchlaufen des Textes
    for index, line in enumerate(text_split):
        
        #Überspringen der Titel- und Innenseite        
        if i > 56:
            
            #Durchlaufen der Städteliste
            for city in city_names:
                
                #Überprüfen ob das erste Element der Zeile komplett mit dem Stadtnamen übereinstimmt
                if city == line.split(" ")[0]:
                    
                    #Check ob in der nächsten Zeile Koordinaten vorkommen
                    #Zeile muss kürzer als 50 Zeichen sein, um Fließtext auszuschließen                                                          
                    if "miles:" in text_split[index+1] and len(text_split[index+1]) < 50:
                        
                        #Erneute Überprüfung ob die nächste Zeile Koordinaten enthält                        
                        if bool(re.search(r'[°]',  text_split[index+1])):
                            
                            #Auswahl der Koordinatenzeile
                            next_line=(text_split[index+1])
                            
                            #Aufteilen der Zeile in 3 Elemente anhand der Doppelpunkte und Auswahl des dritten Objekts
                            population=next_line.split(":")[2]
                            
                            #Entfernen der Klammern                                                        
                            population= (re.sub(r"[\(\)]", "", population))
                            
                            #Ersetzen des Dezimalkommas und Anhängen an die Ergebnisliste
                            population_list.append(population.replace(",","."))
                            
                                                
        else:
            i+=1
            
    return population_list

In [683]:
len(find_city_population(bomber_text))

219

Mit diesen Funktionen haben wir nun alle Meta-Informationen zu den Städten extrahiert. Im Original-Text folgen noch eine Text-Beschreibung der Stadt und eine Auflistung von strategischen Zielen und deren Kategorie.

---
## 4. Export

Alle bis jetzt extrahierten Informationen sollen nun in einer externen Datei gespeichert werden.
### 4.1. XML-Export
Da sich XML sehr gut dafür eignet Daten strukturiert darzustellen, werden unsere Informationen nun in eine XML-Datei umgewandelt.
Der Aufbau der XML-Datei sieht dabei wie folgt aus:

* book
    * title /title
    * subtitle /subtitle
    * part /part
    * range /range
    * preface /preface
    * city (name)
        * state /state
        * coordinates /coordinates
        * distance /distances
        * population /population
    * /city
* /book

In [629]:
#Funktion, die die Ergebnisse an eine vorhandene (manuell erstellte) XML-Datei anfügt
def insert_cities_xml (xml_file):
    
    #Initialisieren des XML-Trees
    tree=ET.parse(xml_file)
    
    #Zuweisen der Wurzel des XML-Trees
    root=tree.getroot()
    
    #Extraktion der Städtenamen
    city_names=find_cities_detailed(bomber_text)
    
    #Extraktion der Bundesländer
    state_list=find_city_state(bomber_text)
    
    #Extraktion der Koordinaten
    coordinates_list=find_city_coordinates(bomber_text)
    
    #Extraktion der Entfernungen zu London
    distances_list=find_city_distances(bomber_text)
    
    #Extraktion der Einwohnerzahlen
    population_list=find_city_population(bomber_text)
    
    #Durchlaufen aller Elemente "book" im XML-Tree
    for book in root.iter('book'):
        
        #Durchlaufen der Städteliste
        for city in city_names: 
                        
            #Anfügen eines neuen Sub-Elements "city" mit dem Stadtnamen als Wert des Attributs "name"
            sub = ET.SubElement(book, 'city')
            sub.set('name', city)
    
    #Durchlaufen aller Elemente "city" im XML-Tree
    for index, city in enumerate (root.iter('city')):        
        
        #Anfügen eines Sub-Elements "state" an das Element "city"
        sub_state = ET.SubElement(city, 'state')
        
        #Check, dass es nicht mehr Elemente "city", als Einträge in der Bundesland-Liste gibt
        if index < len(state_list):
            
            #Das aktuelle Element "state" bekommt als Text-Wert den passenden Eintrag aus der Bundesland-Liste
            sub_state.text = state_list[index]
            
        #Anfügen eines Sub-Elements "coordinates" an das Element "city"
        sub_coordinates = ET.SubElement(city, 'coordinates')
        
        #Check, dass es nicht mehr Elemente "city", als Einträge in der Koordinaten-Liste gibt
        if index < len(coordinates_list):
            
            #Das aktuelle Element "coordinates" bekommt als Text-Wert den passenden Eintrag aus der Koordinaten-Liste
            sub_coordinates.text = coordinates_list[index]
        
        #Anfügen eines Sub-Elements "distance" an das Element "city"
        sub_distances = ET.SubElement(city, 'distance')
        
        #Check, dass es nicht mehr Elemente "city", als Einträge in der Entfernungen-Liste gibt
        if index < len(distances_list):
            
            #Das aktuelle Element "distances" bekommt als Text-Wert den passenden Eintrag aus der Entfernungen-Liste
            sub_distances.text = distances_list[index]
        
        #Anfügen eines Sub-Elements "population" an das Element "city"
        sub_population = ET.SubElement(city, 'population')
        
        #Check, dass es nicht mehr Elemente "city", als Einträge in der Einwohnerzahlen-Liste gibt
        if index < len(population_list):
            
            #Das aktuelle Element "population" bekommt als Text-Wert den passenden Eintrag aus der Einwohnerzahlen-Liste
            sub_population.text = population_list[index]
            
        
      
        
    #Schreiben des aktuellen Trees in die XML-Datei
    tree.write("bomber_output.xml")
        
    

In [684]:
#Aufrufen der Funktion um eine XML-Datei zu erstellen
insert_cities_xml("bomber_xml")

---
### 4.2. CSV-Export

Um unsere Daten graphisch auf https://geobrowser.de.dariah.eu/edit/index.html darstellen zu können, kann hier noch eine .CSV-Datei erstellt werden, die Städtenamen und Koordinaten enthält. Diese kann nach der Erstellung direkt auf die Website hochgeladen werden, um die Punkte auf einer Karte betrachten zu können.

In [631]:
#Funktion zum Export als CSV-Datei
def write_csv():
    
    #Öffnen der Datei, in die die Ergebnisse geschrieben werden
    with open('bomber.csv', mode='w') as bomber_out:
        
        #Initialisieren des CSV-Writers
        csv_writer = csv.writer(bomber_out, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        #Extraktion der Städtenamen
        city_names=find_cities_detailed(bomber_text)
        
        #Extraktion der Koordinaten
        coordinates_list=find_city_coordinates(bomber_text)
        
        #Speichern der Spalten-Überschriften (Vorgabe von DARIAH)
        csv_writer.writerow(['Address', 'Longitude', 'Latitude', 'TimeSpan:begin', 'TimeSpan:end', 'GettyID', 'TimeStamp'])
        
        #Durchlaufen der Städtenamen
        for index, city in enumerate(city_names):
            
            #Aufteilen der passenden Koordinaten in Längen- und Breitengrad
            latitude=coordinates_list[index].split(",")[0]
                
            longitude=coordinates_list[index].split(",")[1]
            
            #Schreiben der Informationen in die CSV-Datei
            csv_writer.writerow([city, longitude, latitude, '', '', '', ''])

In [685]:
write_csv()

---